In [278]:
import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt
import missingno


cf.go_offline(connected = True)

In [279]:
df = pd.read_csv('물품_계약_구성업체_내역.csv')

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning:

Columns (2,6,9,10,12,16,17,18,30,31,34,35) have mixed types.Specify dtype option on import or set low_memory=False.



In [280]:
df.shape

(274924, 38)

In [281]:
df_2 = df.copy()

In [282]:
df_2['등록유형'].unique()

array(['나라장터(G2B)', '자체전자조달시스템'], dtype=object)

In [283]:
df_2 = df_2[df_2['등록유형'] == '나라장터(G2B)']

In [284]:
df_2.columns

Index(['등록유형', '조달구분', '공고시스템명', '계약구분', '계약번호', '계약변경차수', '최종계약여부', '수요기관명',
       '수요기관코드', '수요기관구분', '수요기관지역명', '계약건명', '계약방법', '대표물품분류번호', '대표품명',
       '대표세부물품분류번호', '대표세부품명', '다수공급자계약여부', '우수제품여부', '최초계약일자', '최초계약금액',
       '계약일자', '계약수량', '계약금액', '증감계약수량', '증감계약금액', '공동계약이행방식', '지분율', '업체명',
       '업체사업자등록번호', '업체기업구분명', '입찰공고번호', '입찰공고차수', '입찰계약방법', '낙찰자결정방법',
       '초년도계약번호', '장기계약차수', '장기계속계약여부'],
      dtype='object')

In [285]:
df_2 = df_2.reset_index()

In [286]:
df_3 = df_2[['조달구분','계약구분','계약변경차수','최종계약여부','수요기관구분','계약방법','다수공급자계약여부', '우수제품여부',
            '최초계약금액', '계약수량', '계약금액', '증감계약수량', '증감계약금액', '공동계약이행방식', '지분율',
             '입찰계약방법', '낙찰자결정방법','장기계속계약여부','index']].copy()

In [287]:
df_3.shape

(245665, 19)

In [288]:
df_3.isnull().sum()
# Null 데이터가 너무 많으면 제거한다..
# '공동계약이행방식'인 경우 새로운 데이터를 넣었을 때 코드를 추가하여야 한다.

조달구분              0
계약구분              0
계약변경차수            0
최종계약여부            0
수요기관구분            0
계약방법              0
다수공급자계약여부    201368
우수제품여부            0
최초계약금액            0
계약수량              0
계약금액              0
증감계약수량            0
증감계약금액            0
공동계약이행방식        610
지분율               0
입찰계약방법       138410
낙찰자결정방법      140007
장기계속계약여부          0
index             0
dtype: int64

In [212]:
df_3['다수공급자계약여부'].unique()

array(['N', nan, 'Y'], dtype=object)

In [213]:
df_3['공동계약이행방식'].unique()

array(['분담', '단독', nan, '공동', '공동+분담'], dtype=object)

In [306]:
# df_4 = df_3.loc[(df_3['다수공급자계약여부'].notnull()) &  (df_3['입찰계약방법'].notnull()) & (df_3['낙찰자결정방법'].notnull())]
df_4 = df_3.loc[:,~df_3.columns.isin(['다수공급자계약여부','입찰계약방법','낙찰자결정방법'])]

In [307]:
df_4.head(2)

,조달구분,계약구분,계약변경차수,최종계약여부,수요기관구분,계약방법,우수제품여부,최초계약금액,계약수량,계약금액,증감계약수량,증감계약금액,공동계약이행방식,지분율,장기계속계약여부,index
0,자체조달,총액계약,2,Y,지자체,수의계약,N,11470000.0,902.0,11162100.0,115.0,1423100.0,단독,100.0,N,0
1,자체조달,총액계약,1,N,지자체,제한경쟁,N,101670000.0,1.0,101670000.0,0.0,0.0,단독,100.0,N,1


In [308]:
df_4.isnull().sum()

조달구분          0
계약구분          0
계약변경차수        0
최종계약여부        0
수요기관구분        0
계약방법          0
우수제품여부        0
최초계약금액        0
계약수량          0
계약금액          0
증감계약수량        0
증감계약금액        0
공동계약이행방식    610
지분율           0
장기계속계약여부      0
index         0
dtype: int64

In [309]:
df_4['공동계약이행방식'].value_counts()

단독       243402
공동         1046
분담          532
공동+분담        75
Name: 공동계약이행방식, dtype: int64

In [310]:
df_4 = df_4[df_4['공동계약이행방식'].notnull()]

In [311]:
df_4.describe()

,계약변경차수,최초계약금액,계약수량,계약금액,증감계약수량,증감계약금액,지분율,index
count,245055.000000,2.450550e+05,2.450550e+05,2.450550e+05,2.450550e+05,2.450550e+05,245055.000000,245055.000000
mean,0.770709,9.909307e+08,2.421234e+05,1.382681e+09,1.088087e+05,2.899206e+08,99.533782,122820.494624
std,3.370948,1.333432e+10,7.473741e+06,1.634132e+10,5.814544e+06,7.072978e+09,5.697289,70909.613671
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.312774e+08,-3.562850e+11,0.000000,0.000000
25%,0.000000,6.003025e+06,1.000000e+00,5.970000e+06,1.000000e+00,2.398000e+06,100.000000,61417.500000
50%,0.000000,1.840000e+07,1.000000e+00,1.824400e+07,1.000000e+00,9.477600e+06,100.000000,122818.000000
75%,0.000000,8.592250e+07,3.000000e+02,8.640000e+07,4.600000e+01,3.885250e+07,100.000000,184237.500000
max,99.000000,1.614648e+12,1.503447e+09,1.614648e+12,1.503447e+09,1.614648e+12,100.000000,245664.000000


In [312]:
df_4['수요기관구분'].unique()

array(['지자체', '기타기관', '공기업', '국가기관', '준정부기관', '지방공기업', '교육기관', '기타공공기관',
       '지자체 출자출연기관'], dtype=object)

In [313]:
df_4['수요기관구분'].value_counts()

지자체           87812
교육기관          59183
기타기관          41515
국가기관          20836
준정부기관         10041
기타공공기관         9694
지방공기업          9153
공기업            4270
지자체 출자출연기관     2551
Name: 수요기관구분, dtype: int64

In [314]:
# df_4['수량당계약금액'] = (df_4['계약금액'] + df_4['증감계약금액'])/ (df_4['계약수량'] +  df_4['증감계약수량'])
df_4['총계약금액'] = (df_4['계약금액'] + df_4['증감계약금액'])
df_4['총갯수']  = (df_4['계약수량'] +  df_4['증감계약수량'])

In [315]:
df_4.describe()

,계약변경차수,최초계약금액,계약수량,계약금액,증감계약수량,증감계약금액,지분율,index,총계약금액,총갯수
count,245055.000000,2.450550e+05,2.450550e+05,2.450550e+05,2.450550e+05,2.450550e+05,245055.000000,245055.000000,2.450550e+05,2.450550e+05
mean,0.770709,9.909307e+08,2.421234e+05,1.382681e+09,1.088087e+05,2.899206e+08,99.533782,122820.494624,1.672602e+09,3.509321e+05
std,3.370948,1.333432e+10,7.473741e+06,1.634132e+10,5.814544e+06,7.072978e+09,5.697289,70909.613671,2.064796e+10,1.260367e+07
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.312774e+08,-3.562850e+11,0.000000,0.000000,-1.246840e+11,-2.312774e+08
25%,0.000000,6.003025e+06,1.000000e+00,5.970000e+06,1.000000e+00,2.398000e+06,100.000000,61417.500000,1.090000e+07,2.000000e+00
50%,0.000000,1.840000e+07,1.000000e+00,1.824400e+07,1.000000e+00,9.477600e+06,100.000000,122818.000000,3.333190e+07,2.000000e+00
75%,0.000000,8.592250e+07,3.000000e+02,8.640000e+07,4.600000e+01,3.885250e+07,100.000000,184237.500000,1.473523e+08,4.751000e+02
max,99.000000,1.614648e+12,1.503447e+09,1.614648e+12,1.503447e+09,1.614648e+12,100.000000,245664.000000,3.229296e+12,3.006894e+09


In [ ]:
df_5.shape

In [317]:
# (df_4['수량당계약금액'] >= 0) &
df_5 = df_4[ (df_4['총갯수'] >= 0) & (df_4['총계약금액'] >= 0)].copy()
# 모두 데이터 오류이다. 이를 제거한다.

In [320]:
df_5.shape

(242952, 18)

In [318]:
df_5.isnull().sum()

조달구분        0
계약구분        0
계약변경차수      0
최종계약여부      0
수요기관구분      0
계약방법        0
우수제품여부      0
최초계약금액      0
계약수량        0
계약금액        0
증감계약수량      0
증감계약금액      0
공동계약이행방식    0
지분율         0
장기계속계약여부    0
index       0
총계약금액       0
총갯수         0
dtype: int64

In [321]:
df_5.describe()

,계약변경차수,최초계약금액,계약수량,계약금액,증감계약수량,증감계약금액,지분율,index,총계약금액,총갯수
count,242952.000000,2.429520e+05,2.429520e+05,2.429520e+05,2.429520e+05,2.429520e+05,242952.000000,242952.000000,2.429520e+05,2.429520e+05
mean,0.759199,9.913049e+08,2.437672e+05,1.391254e+09,1.121837e+05,3.003142e+08,99.534676,122797.949957,1.691568e+09,3.559509e+05
std,3.357565,1.338417e+10,7.505466e+06,1.641039e+10,5.818780e+06,7.089944e+09,5.690239,70908.196182,2.073253e+10,1.264874e+07
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-7.986727e+07,-3.562850e+11,0.010000,0.000000,0.000000e+00,0.000000e+00
25%,0.000000,6.000000e+06,1.000000e+00,6.006450e+06,1.000000e+00,2.517522e+06,100.000000,61419.750000,1.118000e+07,2.000000e+00
50%,0.000000,1.834231e+07,1.000000e+00,1.834405e+07,1.000000e+00,9.600000e+06,100.000000,122807.500000,3.388000e+07,2.000000e+00
75%,0.000000,8.514000e+07,3.000000e+02,8.654000e+07,5.000000e+01,3.949745e+07,100.000000,184246.250000,1.498952e+08,4.860000e+02
max,99.000000,1.614648e+12,1.503447e+09,1.614648e+12,1.503447e+09,1.614648e+12,100.000000,245664.000000,3.229296e+12,3.006894e+09


In [322]:
df_5.columns

Index(['조달구분', '계약구분', '계약변경차수', '최종계약여부', '수요기관구분', '계약방법', '우수제품여부',
       '최초계약금액', '계약수량', '계약금액', '증감계약수량', '증감계약금액', '공동계약이행방식', '지분율',
       '장기계속계약여부', 'index', '총계약금액', '총갯수'],
      dtype='object')

In [323]:
df_5['공동계약이행방식'].value_counts()

단독       241315
공동         1033
분담          529
공동+분담        75
Name: 공동계약이행방식, dtype: int64

In [324]:
see = pd.DataFrame()
see['columns'] = df_5.columns
see['type'] = [type(i) for i in df_5.iloc[0,:]]

In [325]:
see

,columns,type
0,조달구분,<class 'str'>
1,계약구분,<class 'str'>
2,계약변경차수,<class 'numpy.int64'>
3,최종계약여부,<class 'str'>
4,수요기관구분,<class 'str'>
5,계약방법,<class 'str'>
6,우수제품여부,<class 'str'>
7,최초계약금액,<class 'numpy.float64'>
8,계약수량,<class 'numpy.float64'>
9,계약금액,<class 'numpy.float64'>


## 범주형을 모두 명목형 숫자로 바꾸어준다.
### 이때 숫자가 무엇을 의미하는지 알기 위해서 다 적어주자

In [326]:
# 조달구분 범주형 확인
df_5['조달구분'].value_counts()

자체조달    199302
중앙조달     43650
Name: 조달구분, dtype: int64

In [327]:
procurement = {'중앙조달': 0 ,'자체조달' : 1}
df_5['조달구분'] = df_5['조달구분'].map(procurement)

In [328]:
# 계약구분 범주형 확인
df_5['계약구분'].value_counts()

총액계약     204648
제3자단가     24228
단가계약      14076
Name: 계약구분, dtype: int64

In [329]:
contract = {'제3자단가' : 0 , '총액계약': 1, '단가계약':2}
df_5['계약구분'] = df_5['계약구분'].map(contract)

In [330]:
# 최종계약여부 범주형 확인
df_5['최종계약여부'].value_counts()

Y    207560
N     35392
Name: 최종계약여부, dtype: int64

In [331]:
final_contract_if = {'Y': 0,'N':1 }
df_5['최종계약여부'] = df_5['최종계약여부'].map(final_contract_if)

In [332]:
# 수요기관구분 범주형 확인
df_5['수요기관구분'].value_counts()

지자체           87032
교육기관          58748
기타기관          40968
국가기관          20676
준정부기관          9961
기타공공기관         9663
지방공기업          9117
공기업            4243
지자체 출자출연기관     2544
Name: 수요기관구분, dtype: int64

In [333]:
agency = {'기타기관' : 0, '국가기관': 1, '지자체' : 2, '교육기관': 3, '공기업': 4, '지자체 출자출연기관':5, '지방공기업':6,
       '기타공공기관':7, '준정부기관':8}
df_5['수요기관구분'] = df_5['수요기관구분'].map(agency)

In [334]:
# 계약방법 범주형 확인
df_5['계약방법'].value_counts()

수의계약    175234
제한경쟁     42284
일반경쟁     24431
지명경쟁      1003
Name: 계약방법, dtype: int64

In [335]:
contract_way = {'제한경쟁' : 0 ,'일반경쟁' : 1, '수의계약' : 2, '지명경쟁': 4}
df_5['계약방법'] = df_5['계약방법'].map(contract_way)

In [337]:
# # 다수공급자계약여부 범주형 확인
# df_5['다수공급자계약여부'].value_counts()

In [338]:
# many_contractor_if = {'Y' : 0 ,'N' : 1}
# df_5['다수공급자계약여부'] = df_5['다수공급자계약여부'].map(many_contractor_if)

In [339]:
# 우수제품여부 범주형 확인
df_5['우수제품여부'].value_counts()

N    239360
Y      3592
Name: 우수제품여부, dtype: int64

In [340]:
many_contractor_if = {'Y' : 0 ,'N' : 1}
df_5['우수제품여부'] = df_5['우수제품여부'].map(many_contractor_if)

In [341]:
# 공동계약이행방식 범주형 확인
df_5['공동계약이행방식'].value_counts()

단독       241315
공동         1033
분담          529
공동+분담        75
Name: 공동계약이행방식, dtype: int64

In [342]:
together_contract_way = {'단독' : 0 ,'공동' : 1,'분담' : 2,'공동+분담':3}
df_5['공동계약이행방식'] = df_5['공동계약이행방식'].map(together_contract_way)

In [344]:
# df_5['입찰계약방법'] = [i[0]+')' for i in df_5['입찰계약방법'].str.split(')')]

In [346]:
# # 입찰계약방법 범주형 확인
# df_5['입찰계약방법'].value_counts()

In [348]:
# how_to_contract = {'일반(3자단가)' : 0 ,'수의(총액)' : 1,'제한(총액)' : 2,'제한(3자단가)' : 3,
#                          '수의(단가)' : 4,'일반(총액)' : 5,'제한(단가)' : 6,'일반(단가)' : 7,'지명(총액)' : 8}
# df_5['입찰계약방법'] = df_5['입찰계약방법'].map(how_to_contract)

In [350]:
# df_5['낙찰자결정방법'] = [i[0] for i in df_5['낙찰자결정방법'].str.split('(')]

In [249]:
# 낙찰자결정방법 범주형 확인
# df_5['낙찰자결정방법'].value_counts()


다수공급자계약에의한 낙찰자 결정    1511
수의                    388
중기간 경쟁물품 계약이행능력심사     165
적격심사                  121
제안서적격자중 예가내 최저투찰자      86
협상에 의한 낙찰자 결정          34
제한적최저가                 15
희망수량경쟁                 11
최저가낙찰자결정                6
기타                      2
Name: 낙찰자결정방법, dtype: int64

In [351]:
# how_to_select = {'다수공급자계약에의한 낙찰자 결정' : 0 ,'수의' : 1,'중기간 경쟁물품 계약이행능력심사' : 2,'적격심사' : 3,
#                          '제안서적격자중 예가내 최저투찰자' : 4,'협상에 의한 낙찰자 결정' : 5,'제한적최저가' : 6,
#                  '희망수량경쟁' : 7,'최저가낙찰자결정' : 8, '기타' : 9}
# df_5['낙찰자결정방법'] = df_5['낙찰자결정방법'].map(how_to_select)

In [352]:
# 장기계속계약여부 범주형 확인
df_5['장기계속계약여부'].value_counts()

N    240074
Y      2878
Name: 장기계속계약여부, dtype: int64

In [353]:
long_time_contract_if = {'Y' : 0 ,'N': 1}
df_5['장기계속계약여부'] = df_5['장기계속계약여부'].map(long_time_contract_if)

In [354]:
df_5.columns

Index(['조달구분', '계약구분', '계약변경차수', '최종계약여부', '수요기관구분', '계약방법', '우수제품여부',
       '최초계약금액', '계약수량', '계약금액', '증감계약수량', '증감계약금액', '공동계약이행방식', '지분율',
       '장기계속계약여부', 'index', '총계약금액', '총갯수'],
      dtype='object')

In [356]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242952 entries, 0 to 245664
Data columns (total 18 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   조달구분      242952 non-null  int64  
 1   계약구분      242952 non-null  int64  
 2   계약변경차수    242952 non-null  int64  
 3   최종계약여부    242952 non-null  int64  
 4   수요기관구분    242952 non-null  int64  
 5   계약방법      242952 non-null  int64  
 6   우수제품여부    242952 non-null  int64  
 7   최초계약금액    242952 non-null  float64
 8   계약수량      242952 non-null  float64
 9   계약금액      242952 non-null  float64
 10  증감계약수량    242952 non-null  float64
 11  증감계약금액    242952 non-null  float64
 12  공동계약이행방식  242952 non-null  int64  
 13  지분율       242952 non-null  float64
 14  장기계속계약여부  242952 non-null  int64  
 15  index     242952 non-null  int64  
 16  총계약금액     242952 non-null  float64
 17  총갯수       242952 non-null  float64
dtypes: float64(8), int64(10)
memory usage: 35.2 MB


## 일단, T_square부터 적용시켜보자

In [357]:
def t_square(data ,alpha: float = None):
    
    array_data = np.array(data)

    
    p = array_data.shape[1]
    m = array_data.shape[0]
    n = 1
    col_mean= np.array(data.agg(['mean']))
    mat_cov = data.cov()
    mat_inv_s = np.linalg.inv(mat_cov)  # inverse S
    
    values = []
    for sample in array_data:
        dif = sample - col_mean
        value = np.matmul(np.matmul(dif,mat_inv_s),dif.T).ravel().tolist()  # T-square
        values.append(value)
    
    LCL = 0
    UCL = ( ( p * (m+1) * (m-1) ) / (m * (m - p)) ) * f.ppf((1-alpha), p, m-p)
            
    return values, LCL, UCL 

In [358]:
df_5 = df_5.set_index('index')

In [359]:
t, lcl, ucl =  t_square(df_5.iloc[:,1:],alpha=0.05)

In [360]:
idx_2 = [i for i,j in enumerate(np.array(t).ravel()) if j > ucl]

In [361]:
df_5.iloc[idx_2].index.tolist()

[2,
 3,
 4,
 8,
 10,
 15,
 16,
 18,
 29,
 31,
 33,
 34,
 36,
 40,
 43,
 45,
 46,
 50,
 53,
 57,
 59,
 60,
 61,
 62,
 65,
 70,
 74,
 75,
 87,
 88,
 102,
 107,
 127,
 129,
 130,
 137,
 144,
 158,
 161,
 167,
 181,
 182,
 187,
 192,
 221,
 243,
 252,
 272,
 273,
 279,
 289,
 309,
 312,
 320,
 321,
 328,
 333,
 334,
 340,
 342,
 344,
 347,
 348,
 349,
 354,
 355,
 357,
 369,
 371,
 378,
 389,
 392,
 397,
 431,
 436,
 458,
 478,
 500,
 512,
 539,
 543,
 548,
 550,
 556,
 578,
 580,
 610,
 611,
 616,
 618,
 620,
 624,
 625,
 626,
 627,
 628,
 638,
 639,
 640,
 641,
 642,
 644,
 650,
 653,
 655,
 657,
 658,
 659,
 662,
 667,
 669,
 670,
 672,
 675,
 678,
 681,
 682,
 683,
 684,
 686,
 707,
 708,
 727,
 746,
 754,
 756,
 762,
 764,
 771,
 787,
 805,
 819,
 835,
 845,
 898,
 916,
 936,
 938,
 942,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 953,
 956,
 958,
 959,
 962,
 964,
 970,
 974,
 975,
 977,
 980,
 983,
 987,
 990,
 994,
 998,
 1001,
 1004,
 1011,
 1023,
 1027,
 1029,
 1031,
 1048,
 1053,

In [277]:
df_2.iloc[df_5.iloc[idx_2].index.tolist()].to_csv('test_3.csv',encoding = 'cp949')